<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/getML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y numpy pandas pyarrow getml
!pip install getml==1.5.0 numpy==1.26.4 pyarrow==16.1.0 pandas==2.2.2

import os
os.kill(os.getpid(), 9)

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: pyarrow 18.1.0
Uninstalling pyarrow-18.1.0:
  Successfully uninstalled pyarrow-18.1.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 62.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1ps_Bh93KQRJ4WbnilORmKd1BaWUXtmt1' # train 20%
# folder_id = "1QE55qJT9JrtRM90-aaSAcwiPKtHOhqB-" # train 50% and 80%

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)

Processing folder: 400hz
Processing folder: 100hz
Processing folder: 800hz


In [2]:
import getml
import numpy as np
import pandas as pd
getml.engine.launch()
getml.set_project("sorting-line")

Launching ./getML --allow-push-notifications=true --allow-remote-ips=false --home-directory=/root/.getML --in-memory=true --install=false --launch-browser=true --log=false --project-directory=/root/.getML/projects in /usr/local/lib/python3.11/dist-packages/getml/.getML/getml-community-1.5.0-amd64-linux...
Launched the getML Engine. The log output will be stored in /root/.getML/logs/getml_20250427211140.log


Connected to project 'sorting-line'.

In [3]:
hz = 100
perc = 20 #percentage of train data

In [4]:
train= pd.read_csv(f'{hz}hz-train-feature-{perc}percent-1000files.csv')
train.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)'], inplace = True)
test = pd.read_csv(f'{hz}hz-test-feature-{perc}percent-1000files.csv')
test.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)'], inplace = True)

In [5]:
train

,Uhrzeit,I_In,V_In,I_Out(A),V_Out(V),Temp(°C),fileno,color
0,2024-08-14 19:17:58.406890:406,0.37,8.66,0.23,9.00,35.13,0,0
1,2024-08-14 19:17:58.414666:414,0.39,8.66,0.31,9.00,34.62,0,0
2,2024-08-14 19:17:58.422872:422,0.48,8.66,0.31,9.03,35.13,0,0
3,2024-08-14 19:17:58.435049:435,0.43,8.70,0.27,9.00,34.87,0,0
4,2024-08-14 19:17:58.443382:443,0.46,8.70,0.27,9.03,34.87,0,0
...,...,...,...,...,...,...,...,...
609718,2024-08-26 18:40:24.232560:232,0.37,8.75,0.31,9.03,32.87,3199,3
609719,2024-08-26 18:40:24.240992:240,0.33,8.70,0.31,9.03,32.87,3199,3
609720,2024-08-26 18:40:24.252801:252,0.31,8.75,0.23,9.03,32.62,3199,3
609721,2024-08-26 18:40:24.261297:261,0.37,8.75,0.31,9.00,32.62,3199,3


In [6]:
roles = {
    getml.data.roles.time_stamp: ["Uhrzeit"],
    getml.data.roles.numerical: [
        "I_In",
        "V_In",
        "I_Out(A)",
        "V_Out(V)",
        "Temp(°C)"
    ],
    getml.data.roles.join_key: ["fileno"],
    getml.data.roles.target: ["color"]
}

population_train = getml.data.DataFrame.from_pandas(
    train,
    name="population_train",
    roles=roles
)

population_test = getml.data.DataFrame.from_pandas(
    test,
    name="population_test",
    roles=roles
)

print("DataFrames created successfully!")
print("Train roles:", population_train.roles)
print("Test roles:", population_test.roles)

DataFrames created successfully!
Train roles: join_key:
  - fileno
numerical:
  - I_In
  - V_In
  - I_Out(A)
  - V_Out(V)
  - Temp(°C)
target:
  - color
time_stamp:
  - Uhrzeit
Test roles: join_key:
  - fileno
numerical:
  - I_In
  - V_In
  - I_Out(A)
  - V_Out(V)
  - Temp(°C)
target:
  - color
time_stamp:
  - Uhrzeit


In [12]:
train['color'] = train['color'].astype(int)

In [13]:
test['color'] = test['color'].astype(int)

In [29]:
# from getml import feature_learning as fl
# feature_learners = [
#     getml.feature_learning.Multirel(
#         loss_function=fl.loss_functions.CrossEntropyLoss,
#         num_threads=4
#     )
# ]
# predictor = getml.predictors.LogisticRegression()
# pipe = getml.pipeline.Pipeline(predictors=[predictor])

# pipe.fit(population_train)
# pipe.score(population_test)

In [28]:
# from getml import feature_learning as fl
# feature_learners = [
#     getml.feature_learning.Multirel(
#         loss_function=fl.loss_functions.CrossEntropyLoss,
#         num_threads=4
#     )
# ]
# predictor = getml.predictors.XGBoostClassifier(objective="multi:softmax")
# pipe = getml.pipeline.Pipeline(predictors=[predictor])

# pipe.fit(population_train)
# pipe.score(population_test)

In [27]:
# from getml import predictors

# predictor = predictors.FastPropClassifier()

# pipe = getml.pipeline.Pipeline(
#     feature_learners=feature_learners,
#     predictors=[predictor]
# )

# pipe.fit(population_train)
# pipe.score(population_test)
